Autor: Karol Szwed\
Data: 17.02.2023 r.\
Projekt: Końcowe zadanie zaliczeniowe - analiza danych z użyciem API dla ZTM w Warszawie

# Cel projektu

TODO: Opisać

# Pobranie danych GPS dla autobusów

TODO: Opisać

In [ ]:
import warsaw_data_api
import datetime
import csv
import time

filename = "Buses_location_afternoon.csv"
_MY_API_KEY = "2620c061-1099-44d9-baab-fdc3a772ab29"  # my api key
ztm = warsaw_data_api.ztm(apikey=_MY_API_KEY)  # pass api key

start_time = time.time()

timespan = 3600  # 60 minutes
time_delta_for_buses = 60  # 1 minute
update_interval = 60  # 1 minute

print("Starting the data import at", datetime.datetime.now())
print("Expected to end the data import at", datetime.datetime.now() + datetime.timedelta(seconds=timespan))

with open(filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["lines", "latitude", "longitude", "time", "brigade", "vehicle_number"])  # write header

while True:
    try:
        buses_all = ztm.get_buses_location()
        with open(filename, 'a', newline='') as file:
            writer = csv.writer(file)
            for bus in buses_all:
                now = datetime.datetime.now()
                time_diff = now - bus.time
                
                # We want to gather data that is current, so we only collect location data
                # that is at most 1 min old
                if time_diff.seconds < time_delta_for_buses:
                    writer.writerow([bus.lines, bus.location.latitude, bus.location.longitude,
                                     bus.time.time(), bus.brigade, bus.vehicle_number])
        time.sleep(update_interval)  # wait for 1 minute between updates
        if time.time() - start_time > timespan:  # If timespan have passed, break the loop
            break
    except Exception:  # If an error occurs, ignore it and try again
        continue

# Pobranie danych o przystankach autobusowych

TODO: Opisać

In [1]:
import json
import pandas as pd
import warsaw_data_api
import csv

file_json = open("bus_stops.json", "r")
json_dict = json.loads(file_json.read())

# Extract the list of dictionaries
dict_list = json_dict['result']

# Convert each dictionary in the list
new_dict_list = []
for d in dict_list:
    new_dict = {item['key']: item['value'] for item in d['values']}
    new_dict_list.append(new_dict)

# Convert the list of new dictionaries into a DataFrame
bus_stops_table = pd.DataFrame(new_dict_list)

# Drop the last column
last_column = bus_stops_table.columns[-1]
bus_stops_table = bus_stops_table.drop(last_column, axis=1)

# Rename the columns
bus_stops_table.columns = ["stop_id", "stop_pole", "stop_name",
                           "street_id", "latitude", "longitude", "direction"]

print(bus_stops_table.head())
print(bus_stops_table.shape)

  stop_id stop_pole stop_name street_id   latitude  longitude       direction
0    1001        01  Kijowska      2201  52.248455  21.044827  al.Zieleniecka
1    1001        02  Kijowska      2201  52.249078  21.044443       Ząbkowska
2    1001        03  Kijowska      2201  52.248928  21.044169  al.Zieleniecka
3    1001        04  Kijowska      2201  52.249969  21.041588       Ząbkowska
4    1001        05  Kijowska      1203  52.250319  21.043861  al.Zieleniecka
(7058, 7)


## Tworzymy słownik zawierający zbiór informację o trasie każdej z linii

Todo: opis

In [3]:
_MY_API_KEY = "2620c061-1099-44d9-baab-fdc3a772ab29"  # my api key
ztm = warsaw_data_api.ztm(apikey=_MY_API_KEY)  # pass api key

bus_line_stops = {}
every_200th = 0 # For printing every 200th entry
for index, row in bus_stops_table.iterrows():
    every_200th += 1
    stop_id = row['stop_id']
    stop_pole = row['stop_pole']
    stop_info = tuple(row.values)
    bus_lines = ztm.get_lines_for_bus_stop_id(stop_id, stop_pole)

    if every_200th == 200:
        print(stop_info)
        print(bus_lines)
        every_200th = 0

    for bus_line in bus_lines:
        if bus_line not in bus_line_stops:
            bus_line_stops[bus_line] = set()
        bus_line_stops[bus_line].add(stop_info)

print(bus_line_stops['504'])

('1055', '01', 'Gorzykowska', '1909', '52.268593', '21.060081', 'Piotra Skargi')
['120', '160', '190', 'N02', 'N11', 'N61']
('1129', '01', 'Bruszewska', '1839', '52.327496', '21.028882', 'Zakłady Zbożowe')
['326', '705', '735', '736']
('1209', '02', 'Powstańców', '1757', '52.277575', '21.115400', 'Czapelska')
['145']
('1296', '01', 'Mochtyńska', '2960', '52.357728', '21.035482', 'Płochocińska')
['326', 'N14']
('1407', '02', 'Wenecka', '0137', '52.356885', '21.133208', 'Struga')
['140', '738', 'L40', 'L45', 'N61']
('1530', '01', 'Januszewicza', '1799', '52.314983', '21.178599', 'Szymańskiego')
['L27']
('1749', '03', 'Piłsudskiego-Kościół', '1757', '52.350916', '21.230183', 'Lipowa')
['L36']
('1864', '01', 'Andersena', '0926', '52.282917', '21.139519', 'Powstańców')
['199']
('2006', '03', 'Międzyborska', '0802', '52.245864', '21.075239', 'Praga-Płd.-Ratusz')
['3', '6', '9', '22', '24', '26']
('2069', '01', 'Pocisk', '1515', '52.252753', '21.151553', 'Marsa-Las')
['183', 'N21']
('2143', '

In [4]:
# Write the table as a pickle file
import pickle
with open('bus_line_stops.pkl', 'wb') as f:
    pickle.dump(bus_line_stops, f)

In [6]:
read_from_pickle_bus_line_stops = None
# Read it again
with open('bus_line_stops.pkl', 'rb') as f:
    read_from_pickle_bus_line_stops = pickle.load(f)
    
print(read_from_pickle_bus_line_stops['189'])

{('5050', '01', 'os.Górczewska', '1303', '52.238926', '20.896240', 'Człuchowska'), ('R-11', '00', 'ZEA KLESZCZOWA', '1221', '52.196412', '20.922925', 'Łopuszańska'), ('4049', '04', 'Łopuszańska', '1403', '52.196192', '20.926955', 'Zajezdnia Kleszczowa'), ('4091', '01', 'Dojazdowa', '1220', '52.198162', '20.920010', 'Zajezdnia Kleszczowa'), ('5057', '01', 'Synów Pułku', '1807', '52.224670', '20.916363', 'Połczyńska'), ('3033', '55', 'św.Bonifacego', '2103', '52.182450', '21.056840', 'Sobieskiego'), ('3097', '02', 'Noskowskiego', '1608', '52.172782', '21.038712', 'Wałbrzyska'), ('4212', '01', 'Parowcowa', '0509', '52.211792', '20.913640', 'Wałowicka'), ('5049', '02', 'Szobera', '1303', '52.236967', '20.896700', 'os.Górczewska'), ('4037', '04', 'Sasanki', '0901', '52.184058', '20.977725', 'Radarowa'), ('4092', '03', 'Ryżowa', '1220', '52.199564', '20.914849', 'Dojazdowa'), ('3092', '02', 'Klarysewska', '2103', '52.184620', '21.060277', 'Sobieskiego'), ('5048', '01', 'Batalionów Chłopskich